In [1]:
%%writefile main.nf
#!/usr/bin/env nextflow

/* pipeline input parameters, make a dir called sdk_data, upload a few dicoms to test */
dicom_data = "$baseDir/sdk_data/06/8654bc98ec4951b25adbe8a0dde5dc/test_few/*.dcm"

process run_inference {

    memory '5 GB'
    accelerator 1
    
    label 'inference'
    
    input:
    path dicom_files
    
    output:
    stdout emit: inference_log
    path('*.csv'), emit: csv_files
    
    script:
    """
    python /app/inference_modified_for_batch.py --dicom-input $dicom_files --classification-out-csv 'classification_results.csv'  
    """

}

// Define the entry workflow (initial workflow for Nextflow to run)
workflow {
    dicom_files = Channel.fromPath(dicom_data)
    run_inference(dicom_files)
}

Overwriting main.nf


In [2]:
%%writefile nextflow.config

process {
    withLabel: inference {
        executor = 'awsbatch'
        queue = 'qa-brh-planx-pla-net-nf-job-queue-aartiv-40uchicago-2eedu'
        container = 'placeholder'
        containerOptions = "--gpus all"
    }
}

aws {
	batch {
		cliPath = '/home/ec2-user/miniconda/bin/aws'
		jobRole = 'placeholder'
	}
}

workDir = 'placeholder'

docker.enabled = true

Overwriting nextflow.config


In [3]:
!nextflow run main.nf -dsl2

N E X T F L O W  ~  version 22.10.6
Launching `main.nf` [small_easley] DSL2 - revision: 34ca4eebb3
Uploading local `bin` scripts folder to s3://qa-brh-planx-pla-net-nf-366143200747/aartiv-40uchicago-2eedu/tmp/be/fcbd58cf89977c87655e270be685f6/bin
[-        ] process > run_inference -

[-        ] process > run_inference [  0%] 0 of 2

executor >  awsbatch (2)
[36/aa5970] process > run_inference (1) [  0%] 0 of 2

executor >  awsbatch (2)
[36/aa5970] process > run_inference (1) [  0%] 0 of 2

executor >  awsbatch (2)
[71/216d3d] process > run_inference (2) [100%] 2 of 2 ✔
Completed at: 22-Jan-2024 22:35:40
Duration    : 6m 21s
CPU hours   : (a few seconds)
Succeeded   : 2


